In [ ]:
# install dependencies
import sys
!{sys.executable} -m pip install langchain gql requests_toolbelt

In [67]:
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.utilities import GraphQLAPIWrapper
import os

llm = AzureChatOpenAI(
    temperature=0, 
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_version="2023-07-01-preview",
    deployment_name="gpt-4-32k-0613",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type="azure",
)

endpoint = "http://localhost:8080/query"

tools = load_tools(
    ["graphql"],
    graphql_endpoint=endpoint,
    llm=llm
)

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [93]:
import requests
import logging

# Set up logging
logger = logging.getLogger(__name__)
handler = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s %(name)-12s %(levelname)-8s %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

In [ ]:
def get_schema():
    """Query the api for its schema"""
    logger.info("Querying API for its schema...")

    global endpoint
    query = """
    query IntrospectionQuery {
        __schema {
            types {
                name
                kind
                fields {
                    name
                    type {
                        name
                        kind
                        ofType {
                            name
                            kind
                        }
                    }
                }
            }
        }
    }"""
    request = requests.post(endpoint, json={"query": query})
    json_output = request.json()

    # Simplify the schema
    simplified_schema = {}
    for type_info in json_output["data"]["__schema"]["types"]:
        if not type_info["name"].startswith("__"):
            fields = type_info.get("fields")
            if fields is not None and fields is not []:
                simplified_schema[type_info["name"]] = {
                    "kind": type_info["kind"],
                    "fields": ", ".join(
                        [
                            field["name"]
                            for field in fields
                            if not field["name"].startswith("__")
                        ]
                    ),
                }
            else:
                simplified_schema[type_info["name"]] = {
                    "kind": type_info["kind"],
                }

    return simplified_schema

graphql_fields = get_schema()
print(graphql_fields)

In [ ]:
examples = """
## List all dependency packages of alpine
query IsDependencyQ1 {
  IsDependency(isDependencySpec: { package: { type: "guac", name: "alpine" }}) {
    dependentPackage {
      type
      namespaces {
        namespace
        names {
          name
        }
      }
    }
  }
}

### List images that depend on logrus
query IsDependencyQ3 {
  IsDependency(isDependencySpec: {
    package: { type: "guac" }
    dependentPackage: { type: "golang", name: "logrus" }
  }) {
    package {
      namespaces {
        namespace
        names {
          name
        }
      }
    }
  }
}
"""

query = "what images depend on go-spew package?"

prompt = f"""Here are some example queries for the graphql endpoint described below:
{examples}

Answer the following question: {query} in the graphql database that has this schema {graphql_fields}. Do not use ``` to start your query."""

result = agent.run(prompt)
logger.info(f"Result: {result}")
